In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from copy import copy
from datetime import datetime, timedelta

from stonesoup.functions import gm_reduce_single

from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.imm import IMMPredictor
from stonesoup.updater.imm import IMMUpdater
from stonesoup.models.transition.linear import ConstantVelocity, \
    CombinedLinearGaussianTransitionModel, LinearGaussianTimeInvariantTransitionModel

from stonesoup.models.measurement.linear import LinearGaussian

from stonesoup.types.state import GaussianState
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.detection import Detection
from stonesoup.types.state import StateVector, CovarianceMatrix, \
    GaussianMixtureState, WeightedGaussianState
from stonesoup.types.track import Track

from stonesoup.simulator.simple import SingleTargetGroundTruthSimulator

from matplotlib.patches import Ellipse


In [2]:
from emgpb2.states import Gaussian as EMGPB2Gaussian
from emgpb2.states import GMM as EMGPB2GMM
from emgpb2.states import GMMSequence as EMGPB2GMMSequence
from emgpb2.models import LinearModel as EMGPB2LinearModel
from emgpb2.models import ConstantVelocity as EMGPB2ConstantVelocity
from emgpb2.EM import SKFEstimator

Plot function

In [3]:
def plot_cov_ellipse(cov, pos, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the
    ellipse patch artist.
    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """

    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:, order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta,
                    **kwargs)

    ax.add_artist(ellip)
    return ellip


Groundtruth and detections

In [4]:
gt_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(1.0),
                         ConstantVelocity(1.0)))

gt_track = []
gt_time = []
iter_model_1 = 120
iter_model_2 = 80

timestamp_init = datetime.now()
state_init_1 = GaussianState(StateVector([[0], [0], [0], [0]]),
                           CovarianceMatrix(np.diag([0.0, 0.0, 0.0, 0.0])),
                           timestamp=timestamp_init)
gt_generator_1 = SingleTargetGroundTruthSimulator(gt_transition_model_1, state_init_1, number_steps=iter_model_1)

for time, gnd_paths in gt_generator_1.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

gt_transition_model_2 = CombinedLinearGaussianTransitionModel(
                    (ConstantVelocity(8.0),
                    ConstantVelocity(8.0)))
state_init_2 = GaussianState(gt_track[-1].state_vector,
                           CovarianceMatrix(np.diag([0.0, 0.0, 0.0, 0.0])),
                           timestamp=gt_time[-1])
gt_generator_2 = SingleTargetGroundTruthSimulator(gt_transition_model_2, state_init_2, number_steps=iter_model_2)
for time, gnd_paths in gt_generator_2.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

detection_track = []
mesurement_noise_1 = np.diag([0.75 ** 2, 0.75 ** 2])
measurement_model_1 = LinearGaussian(ndim_state=4, mapping=[0, 2], noise_covar=mesurement_noise_1)
for i in range(iter_model_1):
    m_ = Detection(measurement_model_1.function(gt_track[i].state_vector, measurement_model_1.rvs(1)), gt_time[i])
    detection_track.append(m_)

mesurement_noise_2 = np.diag([0.5 ** 2, 0.5 ** 2])
measurement_model_2 = LinearGaussian(ndim_state=4, mapping=[0, 2], noise_covar=mesurement_noise_2)
for i in range(iter_model_2):
    ii = i + iter_model_1
    m_ = Detection(measurement_model_2.function(gt_track[ii].state_vector, measurement_model_2.rvs(1)), gt_time[ii])
    detection_track.append(m_)

# print(gt_track)
# print(detection_track)
# print(gt_time)

Filter model. Testing without EMGPB2 estimated parameters.

In [5]:
ft_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(1.0),
                         ConstantVelocity(1.0)))
ft_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (ConstantVelocity(15.0),
                         ConstantVelocity(15.0)))
ft_measurement_noise = np.diag([3.0 ** 2, 3.0 ** 2])

ft_measurement_model = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=ft_measurement_noise)

ft_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
ft_predictor_1 = KalmanPredictor(ft_transition_model_1)
ft_predictor_2 = KalmanPredictor(ft_transition_model_2)
ft_imm_predictor = IMMPredictor([ft_predictor_1, ft_predictor_2], ft_model_transition_matrix)

ft_updater = KalmanUpdater(ft_measurement_model)
ft_imm_updater = IMMUpdater([ft_updater, ft_updater], ft_model_transition_matrix)

ft_state_init = WeightedGaussianState(StateVector([[0], [0], [0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0, 5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)-1):
    measurement = detection_track[i]
    # State prediction
    prediction = ft_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = ft_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = ft_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 1.1001955731652486


Filter model. Testing with correct parameters.

In [6]:
model_transition_matrix = np.array([[0.5, 0.5],
                                    [0.5, 0.5]])

predictor_1 = KalmanPredictor(gt_transition_model_1)
predictor_2 = KalmanPredictor(gt_transition_model_2)
imm_predictor = IMMPredictor([predictor_1, predictor_2],
                             model_transition_matrix)

updater_1 = KalmanUpdater(measurement_model_1)
updater_2 = KalmanUpdater(measurement_model_2)
imm_updater = IMMUpdater([updater_1, updater_2], model_transition_matrix)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)-1):
    measurement = detection_track[i]
    # State prediction
    prediction = imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print('----------------------')
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.8693389892500112


EMGPB2

In [7]:
F = np.asarray([
        [1, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 1],
        [0, 0, 0, 1]
    ])
H = np.asanyarray([
    [1, 0, 0, 0],
    [0, 0, 1, 0]
])
def get_Q(Q_sig, dt=1):
    Q = (Q_sig ** 2) * np.asarray([
        [(1/3)*np.power(dt, 3), (1/2)*np.power(dt, 2), 0, 0],
        [(1/2) * np.power(dt, 2), dt, 0, 0],
        [0, 0, (1/3)*np.power(dt, 3), (1/2)*np.power(dt, 2)],
        [0, 0, (1/2) * np.power(dt, 2), dt]
    ])
    return Q

In [8]:
g1 = EMGPB2Gaussian(np.zeros([4, 1]), 10.0*np.eye(4))
g2 = EMGPB2Gaussian(np.zeros([4, 1]), 10.0*np.eye(4))
initial_gmm_state = EMGPB2GMM([g1, g2])

# measurements = 5 * np.random.randn(200, 2, 1) + 1

detection_track_EMGPB2 = [ele.state_vector for ele in detection_track]
gmmsequence = EMGPB2GMMSequence(np.float64(detection_track_EMGPB2), initial_gmm_state)

m1 = EMGPB2LinearModel(F, get_Q(1.0), H, (3.0 ** 2)*np.eye(2))
m2 = EMGPB2LinearModel(F, get_Q(15.0), H, (3.0 ** 2)*np.eye(2))
initial_models = [m1, m2]

Z = np.ones([2, 2]) / 2
dataset = [gmmsequence]

new_models_all, Z, dataset, LL = SKFEstimator.EM(dataset, initial_models, Z,
                                    max_iters=50, threshold=1e-5, learn_H=True, learn_R=True,
                                    learn_A=True, learn_Q=True, learn_init_state=False, learn_Z=True,
                                    diagonal_Q=False, wishart_prior=False)

new_models = new_models_all[-1]

new_models[0].Q = (new_models[0].Q + new_models[0].Q.T)/2
new_models[1].Q = (new_models[1].Q + new_models[1].Q.T)/2

-1184.5196163564146
model -- 0 new_A: 
[[ 1.00062218e+00  7.94893961e-01 -6.91897894e-03  1.28205251e-02]
 [ 2.77386487e-04  7.93704965e-01 -7.52285647e-03  1.09667063e-02]
 [ 1.76844991e-03  8.60606644e-03  1.00555943e+00  7.97396557e-01]
 [ 1.77786178e-03  2.91599672e-03  5.39875426e-03  7.98154553e-01]]
model -- 1 new_A: 
[[ 1.00059250e+00  8.14535197e-01 -6.42343546e-03  1.81129266e-02]
 [ 1.99828959e-04  7.98541539e-01 -7.61361028e-03  1.45017673e-02]
 [ 1.67354259e-03 -8.11610622e-03  1.00512690e+00  7.72168625e-01]
 [ 1.92384172e-03 -3.82416012e-03  5.82437553e-03  7.59922337e-01]]
model -- 0 new_Q: 
[[4.76708542 3.74491569 0.11630043 0.13247382]
 [3.74491569 6.41110957 0.09900593 0.16880725]
 [0.11630043 0.09900593 4.81769963 3.77315895]
 [0.13247382 0.16880725 3.77315895 6.41558138]]
model -- 1 new_Q: 
[[ 5.61529449  4.48721398 -0.0831948   0.0458333 ]
 [ 4.48721398  7.69906015  0.02458949  0.13965784]
 [-0.08319479  0.02458949  5.67984351  4.45170507]
 [ 0.0458333   0.1396578

-878.1282214437055
model -- 0 new_A: 
[[ 1.00057047e+00  7.99706342e-01 -6.87678461e-03  1.64790483e-02]
 [-4.77604961e-04  9.18750407e-01 -3.80661520e-03 -2.36059273e-03]
 [ 1.66032577e-03  1.22126138e-02  1.00559668e+00  7.94043409e-01]
 [ 7.38017948e-04 -1.07890896e-02  1.98645414e-03  9.08713256e-01]]
model -- 1 new_A: 
[[ 1.00058223e+00  8.02323711e-01 -6.76669043e-03  1.60146451e-02]
 [-4.73336736e-04  9.20639375e-01 -3.73761562e-03 -2.50771471e-03]
 [ 1.63791007e-03  1.02275386e-02  1.00548408e+00  7.94636965e-01]
 [ 7.26466189e-04 -1.27261806e-02  1.89648453e-03  9.09076886e-01]]
model -- 0 new_Q: 
[[1.93334589 1.72056869 0.10078306 0.20419126]
 [1.72056945 3.56845284 0.16645911 0.32469924]
 [0.10082528 0.16652558 2.18403295 1.92013052]
 [0.20425851 0.32485799 1.92013346 3.94837253]]
model -- 1 new_Q: 
[[1.94537123 1.72890584 0.09059632 0.19919493]
 [1.72890662 3.5767255  0.1626297  0.32278614]
 [0.09063877 0.16269635 2.19539959 1.92462996]
 [0.19926241 0.32294507 1.92463287 3.

-860.9934693479319
model -- 0 new_A: 
[[ 1.00050049e+00  7.93029093e-01 -7.23412832e-03  1.90590993e-02]
 [-5.48482136e-04  9.23036485e-01 -3.66821069e-03 -6.29287411e-03]
 [ 1.70057113e-03  2.09032820e-02  1.00599753e+00  7.92497359e-01]
 [ 7.61032181e-04 -8.83286816e-03  2.10068015e-03  9.08414615e-01]]
model -- 1 new_A: 
[[ 1.00074056e+00  8.19720283e-01 -5.87147625e-03  1.14401960e-02]
 [-4.19262136e-04  9.39444134e-01 -2.89449866e-03 -9.24014964e-03]
 [ 1.38640415e-03 -7.35945864e-03  1.00440625e+00  8.01555246e-01]
 [ 6.19916231e-04 -3.16977674e-02  1.03342952e-03  9.13078232e-01]]
model -- 0 new_Q: 
[[1.51397422 1.58241552 0.12006545 0.29589273]
 [1.58239483 3.3721829  0.25132277 0.49795635]
 [0.12040448 0.25149128 1.8207475  1.89400972]
 [0.29604674 0.49823671 1.89403392 4.01064173]]
model -- 1 new_Q: 
[[ 1.66081381  1.66758379 -0.03620519  0.22995244]
 [ 1.66756357  3.44265052  0.18505903  0.46594256]
 [-0.03586059  0.18523122  1.99428678  1.94965662]
 [ 0.23011088  0.46622572

-854.3108643226857
model -- 0 new_A: 
[[ 1.00049954e+00  7.75837479e-01 -7.85205740e-03  2.28391734e-02]
 [-5.19174507e-04  9.05553289e-01 -4.13862129e-03 -9.57245542e-03]
 [ 1.76273115e-03  2.94945306e-02  1.00647669e+00  7.87113488e-01]
 [ 8.64015915e-04  2.77797261e-03  2.72545135e-03  9.02866032e-01]]
model -- 1 new_A: 
[[ 1.00091700e+00  8.56354340e-01 -4.47505693e-03  1.15770018e-02]
 [-2.93478778e-04  9.74709261e-01 -1.58789473e-03 -1.28646197e-02]
 [ 1.02531703e-03 -3.37210734e-02  1.00293657e+00  8.02684933e-01]
 [ 4.44723980e-04 -6.44452032e-02 -3.61048884e-04  9.11222936e-01]]
model -- 0 new_Q: 
[[1.46358445 1.5876474  0.08865267 0.32629268]
 [1.58760128 3.32393905 0.26837996 0.64261898]
 [0.08926763 0.26874329 1.84202722 2.00764845]
 [0.32661055 0.64329043 2.00769987 4.15994391]]
model -- 1 new_Q: 
[[ 1.96839247  1.9959968  -0.31916007  0.11478458]
 [ 1.99595138  3.75251995  0.05233535  0.53609093]
 [-0.3185086   0.05272908  2.2726208   2.20778004]
 [ 0.1151342   0.5367883 

-851.9193088257554
model -- 0 new_A: 
[[ 1.00052969e+00  7.64823083e-01 -8.25550341e-03  2.78921791e-02]
 [-5.09159548e-04  8.97825764e-01 -4.33645437e-03 -1.14683477e-02]
 [ 1.76828263e-03  3.08884223e-02  1.00662237e+00  7.81715165e-01]
 [ 9.15757080e-04  2.25406705e-03  2.77416973e-03  9.02970946e-01]]
model -- 1 new_A: 
[[ 1.00110615e+00  8.76417605e-01 -3.72723718e-03  1.93121479e-02]
 [-1.75477080e-04  9.98911815e-01 -6.91160893e-04 -1.32852382e-02]
 [ 7.43805382e-04 -4.77706637e-02  1.00218198e+00  7.94407618e-01]
 [ 2.76671326e-04 -7.59180493e-02 -9.73665351e-04  9.07589813e-01]]
model -- 0 new_Q: 
[[ 1.55256578e+00  1.66632876e+00 -1.99718978e-05  2.94562781e-01]
 [ 1.66625455e+00  3.29045398e+00  2.22535427e-01  7.25710575e-01]
 [ 9.66102183e-04  2.23187657e-01  1.94567181e+00  2.10113974e+00]
 [ 2.95129719e-01  7.26927890e-01  2.10121816e+00  4.20449089e+00]]
model -- 1 new_Q: 
[[ 2.44878671  2.41040985 -0.70561945 -0.08450127]
 [ 2.41033504  4.02318497 -0.21669894  0.507274

-848.9444373096396
model -- 0 new_A: 
[[ 1.00056008e+00  7.54873937e-01 -8.64396494e-03  3.37384178e-02]
 [-5.25605862e-04  8.94964077e-01 -4.40340725e-03 -1.37981507e-02]
 [ 1.75267923e-03  3.07538620e-02  1.00669213e+00  7.76233909e-01]
 [ 9.29205146e-04 -1.74027920e-03  2.58876032e-03  9.07124214e-01]]
model -- 1 new_A: 
[[ 1.00151453e+00  9.03567840e-01 -2.55803597e-03  3.34671000e-02]
 [ 9.18662329e-05  1.03029228e+00  5.94329517e-04 -1.11987939e-02]
 [ 3.05110356e-04 -6.52557923e-02  1.00124099e+00  7.75484861e-01]
 [ 6.42996297e-05 -8.80210303e-02 -1.59860952e-03  8.98034432e-01]]
model -- 0 new_Q: 
[[ 1.65738872  1.74258293 -0.07214723  0.28028863]
 [ 1.74248159  3.23735928  0.18969145  0.81409338]
 [-0.0706751   0.19075362  2.03693118  2.14655841]
 [ 0.28122246  0.81608273  2.14665642  4.17177293]]
model -- 1 new_Q: 
[[ 3.26655389  3.03063994 -1.38579817 -0.42978833]
 [ 3.03053204  4.3951852  -0.68306739  0.35963179]
 [-1.38414409 -0.68184991  3.31264337  2.80676539]
 [-0.4286

-844.5076340466335
model -- 0 new_A: 
[[ 1.00062205e+00  7.46258885e-01 -8.93083014e-03  3.84488649e-02]
 [-5.23022532e-04  8.95863114e-01 -4.29549444e-03 -1.76782340e-02]
 [ 1.70730253e-03  2.81917813e-02  1.00663404e+00  7.71387971e-01]
 [ 8.99444098e-04 -9.20370518e-03  2.18316587e-03  9.14254690e-01]]
model -- 1 new_A: 
[[ 1.00230161e+00  9.53047476e-01 -4.02948258e-04  6.50534327e-02]
 [ 5.86852663e-04  1.08586669e+00  2.84508883e-03 -3.49033412e-03]
 [-4.56983892e-04 -8.47964665e-02  1.00016928e+00  7.22793783e-01]
 [-1.64635822e-04 -9.85833542e-02 -1.90687252e-03  8.62337138e-01]]
model -- 0 new_Q: 
[[ 1.73687968  1.8003729  -0.09446178  0.30299721]
 [ 1.80025114  3.19224617  0.20192265  0.90812416]
 [-0.09241538  0.20348593  2.09170405  2.14611084]
 [ 0.30438758  0.91104547  2.14621378  4.09597301]]
model -- 1 new_Q: 
[[ 4.70957094  3.97464062 -2.71926417 -1.08216313]
 [ 3.97447961  4.9254674  -1.49764029 -0.013714  ]
 [-2.71690585 -1.49582208  4.65512968  3.3795922 ]
 [-1.0804

-838.661047376868
model -- 0 new_A: 
[[ 1.00073114e+00  7.39302460e-01 -9.04969756e-03  4.00098769e-02]
 [-4.71556539e-04  9.00024836e-01 -4.00806051e-03 -2.27575314e-02]
 [ 1.63966399e-03  2.71551065e-02  1.00657918e+00  7.66860338e-01]
 [ 8.19163574e-04 -1.45562600e-02  1.77148763e-03  9.21984837e-01]]
model -- 1 new_A: 
[[ 1.00360105e+00  1.03507471e+00  2.82831269e-03  1.70036489e-01]
 [ 1.16899338e-03  1.16505154e+00  5.70007531e-03  3.64147282e-02]
 [-1.84700923e-03 -1.16108403e-01  9.99016885e-01  5.53366915e-01]
 [-1.08581180e-04 -1.17940276e-01 -1.22442639e-03  7.46895095e-01]]
model -- 0 new_Q: 
[[ 1.77357299  1.85511091 -0.04242871  0.3629776 ]
 [ 1.85498279  3.223002    0.26636951  0.9725348 ]
 [-0.0398254   0.26843716  2.08904978  2.09335769]
 [ 0.36480781  0.97638264  2.09344892  3.98766622]]
model -- 1 new_Q: 
[[ 6.57561298  4.79383914 -4.78162062 -1.93737678]
 [ 4.79350888  5.18743432 -2.39294289 -0.57472481]
 [-4.7787289  -2.39073421  6.87765906  3.9215098 ]
 [-1.93521

-832.7150296328765
model -- 0 new_A: 
[[ 1.00085970e+00  7.32195253e-01 -9.08807623e-03  3.84925378e-02]
 [-4.00347086e-04  9.02286277e-01 -3.79191323e-03 -2.57540159e-02]
 [ 1.57513598e-03  3.15174153e-02  1.00666651e+00  7.62899979e-01]
 [ 7.43461677e-04 -1.36509649e-02  1.62262116e-03  9.26242028e-01]]
model -- 1 new_A: 
[[ 1.00501687e+00  1.09882697e+00  3.44930148e-03  4.67265543e-01]
 [ 5.54141257e-04  1.20524439e+00  4.41673727e-03  1.83875609e-01]
 [-5.14886395e-03 -1.46055980e-01  9.97965704e-01  1.72317529e-01]
 [ 3.71969383e-04 -1.21370595e-01  3.05023827e-03  4.74721763e-01]]
model -- 0 new_Q: 
[[1.78285109 1.92559864 0.04739258 0.41685898]
 [1.92547428 3.34292421 0.319577   0.95623938]
 [0.05046293 0.32211019 2.02661553 2.0197221 ]
 [0.41903763 0.96096114 2.0198009  3.92222918]]
model -- 1 new_Q: 
[[ 4.81053366  3.64096475 -2.24391518 -0.6871044 ]
 [ 3.64014137  4.46150317 -0.88775551 -0.68434819]
 [-2.2413304  -0.88605213  2.97393326  1.06843219]
 [-0.68514696 -0.68069821

Filter model. Testing with learnt parameters from EMGPB2.

In [9]:
em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A, covariance_matrix=new_models[0].Q)
em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A, covariance_matrix=new_models[1].Q)
em_measurement_noise_1 = new_models[0].R
em_measurement_noise_2 = new_models[1].R

em_measurement_model_1 = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=em_measurement_noise_1)
em_measurement_model_2 = LinearGaussian(ndim_state=4, mapping=[0, 2],
                                   noise_covar=em_measurement_noise_2)

em_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
em_predictor_1 = KalmanPredictor(em_transition_model_1)
em_predictor_2 = KalmanPredictor(em_transition_model_2)
em_imm_predictor = IMMPredictor([em_predictor_1, em_predictor_2], em_model_transition_matrix)

em_updater_1 = KalmanUpdater(em_measurement_model_1)
em_updater_2 = KalmanUpdater(em_measurement_model_2)
em_imm_updater = IMMUpdater([em_updater_1, em_updater_2], em_model_transition_matrix)

em_state_init = WeightedGaussianState(StateVector([[0], [0], [0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0, 5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(em_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)-1):
    measurement = detection_track[i]
    
    # State prediction
    prediction = em_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = em_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = em_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(em_measurement_model_1.function(track[i].state_vector, np.zeros((2,1))) - em_measurement_model_1.function(gt_track[i].state_vector, np.zeros((2,1))))
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.8839332824544972
